In [1]:
import pandas as pd
import numpy as np
from lxml import etree
import xml.etree.ElementTree as ET
import time
import math
import pylab as pl
import matplotlib.pyplot as plt
import seaborn as sns # improves plot aesthetics

# Function

In [2]:
class DataReader:
  """
  Main class to read in xml files provided by the DFB.

  Returns:
      ElementTree root: The root of the xml document.

  """

  def __init__(self,xml_file):
    self.xml_root = self._load_data(xml_file)


  def _load_data(self,xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root

  def create_dataframe(self,df_cols=None,findall_string=".//Object",time_search=None):
    """
    Create a dataframe from the xml inputs

    Args:
        df_cols (list): Optional, the column names. If no names given, columns
          detected automatically
        findall_string (str): The xpath string to iterate trees (https://www.w3schools.com/xml/xml_xpath.asp)
        time_search (float): Will currently look at parent if it's an Event, it
          will compare the time_search to the EventTime. I.e. this is a means of getting
          all passes before the frame time.

    Returns:
        pd.dataframe
    """

    populate_column_names=False
    if df_cols is None:
        df_cols = []
        populate_column_names=True

    out_df = pd.DataFrame(columns = df_cols)

    parent_map = dict((c, p) for p in self.xml_root.getiterator() for c in p)
    for node in self.xml_root.findall(findall_string):
        if time_search is not None:
          if (parent_map[node].tag != "Event"): continue
          event_time = parent_map[node].attrib["EventTime"]
          ts = ciso8601.parse_datetime(event_time)
          # to get time in seconds:
          seconds=time.mktime(ts.timetuple())
          # If timestamp is larger than the search, exit
          if seconds > time_search: continue

        # If column names are not provided, get them
        if populate_column_names==True:
            for key,value in node.attrib.items():
                df_cols.append(key)
                populate_column_names=False

        res = []
        for c in df_cols:
            res.append(node.attrib.get(c))
        out_df = out_df.append(pd.Series(res, index = df_cols), ignore_index=True)
    return out_df


In [3]:
def get_tracking_data(xml_positions, half='firstHalf'):
        """Returns a dataframe with the tracking (position: x, y, z) of all players + the ball
        
        Dropping rows where ball positions is NAN

        +-------+-----------+-----------+
        |       |   BALL    |  TeamId   |
        +       +-----------+-----------+
        |       |  BallId   | PlayerId  |
        + N | T +---+---+---+---+---+---+
        |       | X | Y | Z | X | Y | A |
        +=======+===+===+===+===+===+===+
        | 0 | 0 |123|456|789|012|456|789|
        +-------+-----------+-----------+

        :param xml_positions: the parsed xml tracking data (lxml object)
        :param half: the description of the half: either "firstHalf" or "secondHalf", etc.
        :return: a dataframe containing the tracking data: position of all players and the ball
        """
        def change_type(df, frameset):
            """
            change type of columns
            different columns names and tpyes for the ball and for the rest of the players
            """
            if frameset.attrib.get('TeamId') == 'BALL':
                return df.astype(
                    # change types of columns for the ball
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        'Z':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                        'BallPossession':int, 
                        'BallStatus':int,
                    }, 
                    errors='raise'
                )
            else:
                return df.astype(
                    # change types of columns for the players
                    dtype={
                        'A':float, 
                        'D':float, 
                        'M':int, 
                        'N':int, 
                        'S':float, 
                        'X':float, 
                        'Y':float,
                        #'T':'datetime64[ns]', # this breaks the timezone
                    }, 
                    errors='raise'
                )
        

        def create_columns(df, frameset):
            """A utility function to create a multiindex columns"""
            person_id = frameset.attrib.get('PersonId')
            team_id = frameset.attrib.get('TeamId')
            df.columns = pd.MultiIndex.from_product(
                [[team_id], [person_id], df.columns],
                names=['TeamId', 'PersonId', 'Position']
            )
            return df

        tracking_players = pd.concat([
            pd.DataFrame.from_records(
                [dict(frame.attrib) for frame in frameset.getchildren()]
            ).pipe(change_type, frameset=frameset)
             .set_index(['N', 'T'])
             .pipe(create_columns, frameset=frameset)
            for frameset in xml_positions.xpath(F'//Positions/FrameSet[@GameSection = "{half}"]')
        ], axis=1, sort=False)
        
        # this is casting the 'T' values in the index to datetime with the correct time zone
        tracking_players.index.set_levels(pd.to_datetime(tracking_players.index.get_level_values(1)), level=1, inplace = True)

        return tracking_players

In [4]:
def calculateDistance(x1,y1,x2,y2):  
    if np.isnan(x1)==True or np.isnan(y1)==True or np.isnan(x2)==True or np.isnan(y2)==True:
        dist=float('NaN')
    else:
        dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  
    return dist  

In [5]:
def getdirectionofplay(teamid,half):
    if matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].mean(axis=1).iat[0] < 0:
        dirofplay = 'left to right'
    if matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].mean(axis=1).iat[0] > 0:
        dirofplay = 'right to left'
    return  dirofplay

In [6]:
def getteamsheet(teamid,half):
    teamsheet=matchinfo.get(half)["Dataframe"].loc[matchinfo.get(half)["Dataframe"].index.get_level_values(0)[0],(teamid,slice(None),"X")].columns.get_level_values(level=1).unique()
    return teamsheet

# GER EST

In [5]:
path='C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/'    #Privat

In [6]:
xml_positions = etree.parse('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##privat
kpimerged = pd.read_excel('C:/Users/Tim/Desktop/QIndex/data/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##privat

In [7]:
path='C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/'    #Arbeit
xml_positions = etree.parse('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFL_04_03_positions_raw_observed_DFL-COM-000001_DFL-MAT-003BEU.xml') ##Arbeit
kpimerged = pd.read_excel('C:/Users/Jung/Desktop/Q-Index Master/19-06-12_Mainz_GER_EST Sportec Data/DFB_KPI_Merged.xlsx') ##Arbeit

In [9]:
kpimergednew = kpimerged.rename({'FRAME_NUMBER': 'N'}, axis=1)

In [10]:
positions=get_tracking_data(xml_positions)  
positions2=get_tracking_data(xml_positions,"secondHalf") 
##create dataframe from xml input

In [10]:
#positions2=get_tracking_data(xml_positions,"secondHalf") 

In [11]:
CUID1 = 'DFL-CLU-000N8Y'
cuid1_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID1)
cuid1_team_data = DataReader(path+cuid1_player_info)
cuid1_team_meta_df = cuid1_team_data.create_dataframe()
cuid1_player_IDs = cuid1_team_meta_df.ObjectId.tolist()

In [12]:
CUID2 = 'DFL-CLU-000N8Z'
cuid2_player_info = "DFL_01_05_masterdata_{}_DFL-SEA-0001K3_player (1).xml".format(CUID2)
cuid2_team_data = DataReader(path+cuid2_player_info)
cuid2_team_meta_df = cuid2_team_data.create_dataframe()
cuid2_player_IDs = cuid2_team_meta_df.ObjectId.tolist()

In [13]:
matchinfo = {
    CUID1: {
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
    },
    2: {
        "Dataframe":positions2,
    }
}

In [14]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
    },
    1: {
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [15]:
players=positions.columns.get_level_values(level=1).unique()
players=players.append(positions2.columns.get_level_values(level=1).unique())
players=players.unique()
players=players.drop("DFL-OBJ-0000XT")
players
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        positions[CUID1,player,'BP']=0
        positions[CUID1,player,'DtB']=positions.apply(lambda x: calculateDistance(x[CUID1,player,'X'],x[CUID1,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
    if player in matchinfo.get(1)[CUID2]:
        positions[CUID2,player,'BP']=0
        positions[CUID2,player,'DtB']=positions.apply(lambda x: calculateDistance(x[CUID2,player,'X'],x[CUID2,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)

for player in players:
    if player in matchinfo.get(2)[CUID1]:
        positions2[CUID1,player,'BP']=0
        positions2[CUID1,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID1,player,'X'],x[CUID1,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
    if player in matchinfo.get(2)[CUID2]:
        positions2[CUID1,player,'BP']=0
        positions2[CUID2,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID2,player,'X'],x[CUID2,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)

In [98]:
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        positions2[CUID1,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID1,player,'X'],x[CUID1,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)
    if player in matchinfo.get(2)[CUID2]:
        positions2[CUID2,player,'DtB']=positions2.apply(lambda x: calculateDistance(x[CUID2,player,'X'],x[CUID2,player,'Y'],x['BALL','DFL-OBJ-0000XT','X'],x['BALL','DFL-OBJ-0000XT','Y']),axis=1)

In [16]:
##Placeholder Column ob in Ballbesitz oder nicht
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        positions[CUID1,player,'BP']=0
    if player in matchinfo.get(1)[CUID2]:
        positions[CUID2,player,'BP']=0
        
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        positions2[CUID1,player,'BP']=0
    if player in matchinfo.get(2)[CUID2]:
        positions2[CUID2,player,'BP']=0

In [170]:
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        positions2[CUID1,player,'BP']=0
    if player in matchinfo.get(2)[CUID2]:
        positions2[CUID2,player,'BP']=0

In [17]:
balldf=positions.loc[:,("BALL","DFL-OBJ-0000XT",slice(None))]   ##Ballkoordinaten 1. Halbzeit
CUID1df = positions.loc[:,(CUID1,slice(None),slice(None))]     ##Koordinaten von CUID1 1. Halbzeit
cuid1players=CUID1df.columns.get_level_values(level=1).unique()        ##Spieler, die in der 1. Halbzeit auf dem Platz standen
cuid1balldf=pd.merge(CUID1df, balldf, left_index=True, right_index=True)  

CUID2df = positions.loc[:,(CUID2,slice(None),slice(None))]     ##Koordinaten von CUID2 1. Halbzeit
cuid2players=CUID2df.columns.get_level_values(level=1).unique()        ##Spieler, die in der 1. Halbzeit auf dem Platz standen
cuid2balldf=pd.merge(CUID2df, balldf, left_index=True, right_index=True)   ##Gemergter Dataframe Ball und CUID1 1. Halbzeit

balldf2=positions2.loc[:,("BALL","DFL-OBJ-0000XT",slice(None))]   ##Ballkoordinaten 2. Halbzeit
CUID1df2 = positions2.loc[:,(CUID1,slice(None),slice(None))]     ##Koordinaten von CUID1 2. Halbzeit
cuid1players2=CUID1df2.columns.get_level_values(level=1).unique()        ##Spieler, die in der 2. Halbzeit auf dem Platz standen
cuid1balldf2=pd.merge(CUID1df2, balldf2, left_index=True, right_index=True)   ##Gemergter Dataframe Ball und CUID1 2. Halbzeit

CUID2df2 = positions2.loc[:,(CUID2,slice(None),slice(None))]     ##Koordinaten von CUID1 1. Halbzeit
cuid2players2=CUID2df2.columns.get_level_values(level=1).unique()        ##Spieler, die in der 1. Halbzeit auf dem Platz standen
cuid2balldf2=pd.merge(CUID2df2, balldf2, left_index=True, right_index=True)   ##Gemergter Dataframe Ball und CUID1 1. Halbzeit

In [18]:
matchinfo = {
    CUID1: {
        1:getdirectionofplay(CUID1,1),
        2:getdirectionofplay(CUID1,2),
        "BallPoss":1,
        "TeamID":CUID1,
        "TeamBallDf1":cuid1balldf,
        "TeamBallDf2":cuid1balldf2,
    },
    CUID2: {
        1:getdirectionofplay(CUID2,1),
        2:getdirectionofplay(CUID2,2),
        "BallPoss":2,
        "TeamID":CUID2,
        "TeamBallDf1":cuid2balldf,
        "TeamBallDf2":cuid2balldf2,
    },
    1: {
        "BallDf":balldf,
        "Dataframe":positions,
        CUID1:getteamsheet(CUID1,1),
        CUID2:getteamsheet(CUID2,1),
        "Startframe":positions.index.get_level_values(0)[0],
        "Endframe":positions.index.get_level_values(0)[positions.shape[0]-1],
    },
    2: {
        "BallDf":balldf2,
        "Dataframe":positions2,
        CUID1:getteamsheet(CUID1,2),
        CUID2:getteamsheet(CUID2,2),
        "Startframe":positions2.index.get_level_values(0)[0],
        "Endframe":positions2.index.get_level_values(0)[positions.shape[0]-1],
    },
    "left to right":{
        "Goalline":52.5
    },
    "right to left":{
        "Goalline":-52.5
    }
}

In [21]:
kpimergednew.loc[1,:]

EVENT_ID                                 10429828
MUID                               DFL-MAT-003BEU
MATCH_DAY                                       1
PUID1                              DFL-OBJ-0027G6
PUID2                              DFL-OBJ-000191
SHIRT_NUMBER1                                  20
SHIRT_NUMBER2                                  18
NOMATCH                                         0
GDCP_EVENT_TIME        2019-06-11 20:47:00.637000
TRACKING_TIME          2019-06-11 20:47:01.920000
X_EVENT                                         0
Y_EVENT                                         0
X_TRACKING                                    0.2
Y_TRACKING                                      0
SUBTYPE                                      Pass
N                                           10008
GDCP_EVENT_ID                      17318500000005
CUID                               DFL-CLU-000N8Y
NORECEIVER                                  False
XRec                                         -7.6


In [33]:
positions.loc[slice(10000,10005),(slice(None),"DFL-OBJ-002FZW","X")]

,TeamId,DFL-CLU-000N8Z
,PersonId,DFL-OBJ-002FZW
,Position,X
N,T,
10000,2019-06-11 20:47:01.600000+02:00,0.58
10001,2019-06-11 20:47:01.640000+02:00,0.60
10002,2019-06-11 20:47:01.680000+02:00,0.60
10003,2019-06-11 20:47:01.720000+02:00,0.60
10004,2019-06-11 20:47:01.760000+02:00,0.60
10005,2019-06-11 20:47:01.800000+02:00,0.60


In [56]:
kpimergednew.loc[1,"NORECEIVER"]==False

True

In [94]:
positions.loc[slice(10000,10010),(slice(None),slice(None),"BP")]

TeamId                                 DFL-CLU-000N8Y                 \
PersonId                               DFL-OBJ-000191 DFL-OBJ-0027G6   
Position                                           BP             BP   
N     T                                                                
10000 2019-06-11 20:47:01.600000+02:00              0              0   
10001 2019-06-11 20:47:01.640000+02:00              0              0   
10002 2019-06-11 20:47:01.680000+02:00              0              0   
10003 2019-06-11 20:47:01.720000+02:00              0              0   
10004 2019-06-11 20:47:01.760000+02:00              0              0   
10005 2019-06-11 20:47:01.800000+02:00              0              0   
10006 2019-06-11 20:47:01.840000+02:00              0              0   
10007 2019-06-11 20:47:01.880000+02:00              0              0   
10008 2019-06-11 20:47:01.920000+02:00              0              1   
10009 2019-06-11 20:47:01.960000+02:00              0              0   
10010 2019-06-11 20:47:02+02:00                     0              0   

TeamId                                 DFL-CLU-000N8Z                 \
PersonId                               DFL-OBJ-002FZX DFL-OBJ-002G01   
Position                                           BP             BP   
N     T                                                                
10000 2019-06-11 20:47:01.600000+02:00              0              0   
10001 2019-06-11 20:47:01.640000+02:00              0              0   
10002 2019-06-11 20:47:01.680000+02:00              0              0   
10003 2019-06-11 20:47:01.720000+02:00              0              0   
10004 2019-06-11 20:47:01.760000+02:00              0              0   
10005 2019-06-11 20:47:01.800000+02:00              0              0   
10006 2019-06-11 20:47:01.840000+02:00              0              0   
10007 2019-06-11 20:47:01.880000+02:00              0              0   
10008 2019-06-11 20:47:01.920000+02:00              0              0   
10009 2019-06-11 20:47:01.960000+02:00              0              0   
10010 2019-06-11 20:47:02+02:00                     0              0   

TeamId                                 DFL-CLU-000N8Y DFL-CLU-000N8Z  \
PersonId                               DFL-OBJ-0000I4 DFL-OBJ-002FZQ   
Position                                           BP             BP   
N     T                                                                
10000 2019-06-11 20:47:01.600000+02:00              0              0   
10001 2019-06-11 20:47:01.640000+02:00              0              0   
10002 2019-06-11 20:47:01.680000+02:00              0              0   
10003 2019-06-11 20:47:01.720000+02:00              0              0   
10004 2019-06-11 20:47:01.760000+02:00              0              0   
10005 2019-06-11 20:47:01.800000+02:00              0              0   
10006 2019-06-11 20:47:01.840000+02:00              0              0   
10007 2019-06-11 20:47:01.880000+02:00              0              0   
10008 2019-06-11 20:47:01.920000+02:00              0              0   
10009 2019-06-11 20:47:01.960000+02:00              0              0   
10010 2019-06-11 20:47:02+02:00                     0              0   

TeamId                                                DFL-CLU-000N8Y  \
PersonId                               DFL-OBJ-002FZP DFL-OBJ-0000SE   
Position                                           BP             BP   
N     T                                                                
10000 2019-06-11 20:47:01.600000+02:00              0              0   
10001 2019-06-11 20:47:01.640000+02:00              0              0   
10002 2019-06-11 20:47:01.680000+02:00              0              0   
10003 2019-06-11 20:47:01.720000+02:00              0              0   
10004 2019-06-11 20:47:01.760000+02:00              0              0   
10005 2019-06-11 20:47:01.800000+02:00              0              0   
10006 2019-06-11 20:47:01.840000+02:0

In [65]:
i=1
positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]

N      T                               
10030  2019-06-11 20:47:02.800000+02:00    0
10031  2019-06-11 20:47:02.840000+02:00    0
10032  2019-06-11 20:47:02.880000+02:00    0
10033  2019-06-11 20:47:02.920000+02:00    0
10034  2019-06-11 20:47:02.960000+02:00    0
10035  2019-06-11 20:47:03+02:00           0
10036  2019-06-11 20:47:03.040000+02:00    0
10037  2019-06-11 20:47:03.080000+02:00    0
10038  2019-06-11 20:47:03.120000+02:00    0
10039  2019-06-11 20:47:03.160000+02:00    0
10040  2019-06-11 20:47:03.200000+02:00    0
10041  2019-06-11 20:47:03.240000+02:00    0
10042  2019-06-11 20:47:03.280000+02:00    0
Name: (DFL-CLU-000N8Y, DFL-OBJ-000191, BP), dtype: int64

In [161]:
#1st try
for i in range(kpimergednew.shape[0]):
    if kpimergednew.loc[i,"SUBTYPE"]=="Pass" and kpimergednew.loc[i,"NORECEIVER"]==False and kpimergednew.loc[i,"EVALUATION"]=="successfullyComplete":
        ##falls Event Pass und es einen abhnehmer gibt, der im eigenen Team ist
        if kpimergednew.loc[i, "N"]<matchinfo.get(2)["Startframe"] and kpimergednew.loc[i+1, "N"]<matchinfo.get(2)["Startframe"]:
            ##Überprüfung welche Halbzeit
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
                positions.loc[slice(kpimergednew.loc[i-1,"N"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
                positions.loc[slice(kpimergednew.loc[i-1,"RECFRM"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
                
        if kpimergednew.loc[i,"N"]>matchinfo.get(2)["Startframe"]:
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
                positions2.loc[slice(kpimergednew.loc[i-1,"N"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions2.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
                positions2.loc[slice(kpimergednew.loc[i-1,"RECFRM"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions2.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
                            
    if kpimergednew.loc[i,"SUBTYPE"]=="BallClaiming":
        if kpimergednew.loc[i, "N"]<matchinfo.get(2)["Startframe"] and kpimergednew.loc[i+1, "N"]<matchinfo.get(2)["Startframe"]:
            positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
        else:
            positions2.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
    
    if kpimergednew.loc[i,"SUBTYPE"]=="SavedShot":
        if kpimergednew.loc[i, "N"]<matchinfo.get(2)["Startframe"] and kpimergednew.loc[i+1, "N"]<matchinfo.get(2)["Startframe"]:
            positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
        else:
            positions2.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
    

In [175]:
#2nd try
for i in range(kpimergednew.shape[0]-1):
    #print(kpimergednew.loc[i,"N"])
    if kpimergednew.loc[i,"SUBTYPE"]=="Pass" and kpimergednew.loc[i,"NORECEIVER"]==False and kpimergednew.loc[i,"EVALUATION"]=="successfullyComplete":
        ##falls Event Pass und es einen abhnehmer gibt, der im eigenen Team ist
        if kpimergednew.loc[i, "N"]<matchinfo.get(2)["Startframe"] and kpimergednew.loc[i+1, "N"]<matchinfo.get(2)["Startframe"]:
            ##Überprüfung welche Halbzeit
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
                positions.loc[slice(kpimergednew.loc[i-1,"N"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
                positions.loc[slice(kpimergednew.loc[i-1,"RECFRM"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
                
        if kpimergednew.loc[i,"N"]>matchinfo.get(2)["Startframe"]:
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
                positions2.loc[slice(kpimergednew.loc[i-1,"N"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions2.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
                positions2.loc[slice(kpimergednew.loc[i-1,"RECFRM"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
                positions2.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
                            
    if kpimergednew.loc[i,"SUBTYPE"]!="Pass" and kpimergednew.loc[i,"SUBTYPE"]!="Substitution":
        if kpimergednew.loc[i, "N"]<matchinfo.get(2)["Startframe"] and kpimergednew.loc[i+1, "N"]<matchinfo.get(2)["Startframe"]:
            positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
        else:
            positions2.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1

In [109]:
len(positions[(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")]==1)])

67675

In [140]:
positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")].values.flatten()

#len(currentdf[(currentdf.BP=="Yes") & (currentdf.FinalThird=="Yes") & (currentdf.Poss=="No")& (currentdf.Forward=="Yes")].index)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [158]:
positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1]

47190

In [159]:
len(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")])

67675

In [143]:
##Anzahl der 0 und 1 in der Spalte BP
pd.value_counts(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")].values.flatten())[0]  ##anzahl frames nicht in ballbesitz
pd.value_counts(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")].values.flatten())[1]  ##anzahl frames in ballbesitz

3667

In [160]:
#Anzahl der 0 und 1 in der Spalte BP
pd.value_counts(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[0]  ##anzahl frames nicht in ballbesitz
pd.value_counts(positions.loc[slice(None),(slice(None),"DFL-OBJ-0002F5","BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]  ##anzahl frames in ballbesitz

2743

In [149]:
positions.shape[0]

67675

In [ ]:
### EST GOALKEEPER DFL-OBJ-002FZX

In [156]:
positions.loc[slice(13032,13500),(slice(None),"DFL-OBJ-002FZX","BP")]

,TeamId,DFL-CLU-000N8Z
,PersonId,DFL-OBJ-002FZX
,Position,BP
N,T,
13032,2019-06-11 20:49:02.880000+02:00,0
13033,2019-06-11 20:49:02.920000+02:00,0
13034,2019-06-11 20:49:02.960000+02:00,0
13035,2019-06-11 20:49:03+02:00,0
13036,2019-06-11 20:49:03.040000+02:00,0
13037,2019-06-11 20:49:03.080000+02:00,0
13038,2019-06-11 20:49:03.120000+02:00,0


In [153]:
((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum()

47190

In [154]:
##1st try
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]
        print(CUID1,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
    if player in matchinfo.get(1)[CUID2]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")].values.flatten())[1]
        print(CUID2,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())

DFL-CLU-000N8Y DFL-OBJ-000191 4.9099385463021825
DFL-CLU-000N8Y DFL-OBJ-0027G6 1.3943632125450307
DFL-CLU-000N8Z DFL-OBJ-002FZX 2.4051705869887687
DFL-CLU-000N8Z DFL-OBJ-002G01 0.743801652892562
DFL-CLU-000N8Y DFL-OBJ-0000I4 0.9578300487391397
DFL-CLU-000N8Z DFL-OBJ-002FZQ 1.2354312354312353
DFL-CLU-000N8Z DFL-OBJ-002FZP 0.13562195380377198
DFL-CLU-000N8Y DFL-OBJ-0000SE 3.371477007840644
DFL-CLU-000N8Z DFL-OBJ-002G03 1.4070777707141344
DFL-CLU-000N8Z DFL-OBJ-002G08 1.042593769866497
DFL-CLU-000N8Y DFL-OBJ-0001HT 3.7995337995337994
DFL-CLU-000N8Y DFL-OBJ-0002AU 1.2100021190930281
DFL-CLU-000N8Y DFL-OBJ-0000OJ 5.2998516634880275
DFL-CLU-000N8Y DFL-OBJ-0000ZS 4.3399025217207035
DFL-CLU-000N8Y DFL-OBJ-0002F5 7.770714134350498
DFL-CLU-000N8Z DFL-OBJ-002FZY 0.4598431871159144
DFL-CLU-000N8Z DFL-OBJ-002G05 0.8243271879635516
DFL-CLU-000N8Y DFL-OBJ-0000O3 1.2078830260648443
DFL-CLU-000N8Z DFL-OBJ-002FZW 0.5615596524687434
DFL-CLU-000N8Y DFL-OBJ-00258K 4.305997033269761
DFL-CLU-000N8Z DFL-OBJ-0

In [167]:
#2nd try
summe=0
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]
        prozanteil=(anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID1,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)
    if player in matchinfo.get(1)[CUID2]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")].values.flatten())[1]
        prozanteil=(anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID2,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)

4.920534011443102
6.314897223988133
8.720067810976902
9.463869463869464
10.421699512608605
11.65713074803984
11.792752701843613
15.164229709684257
16.571307480398392
17.613901250264888
21.413435049798686
22.640389913117186
27.940241576605214
32.28014409832592
38.14155541428269
38.60139860139861
39.42572578936216
40.70353888535708
41.26509853782582
45.57109557109558
47.12015257469804
47.652044924772206


In [185]:
#simplester ansatz
summe=0
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]
        prozanteil=(anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID1,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)
    if player in matchinfo.get(1)[CUID2]:
        anzframes=pd.value_counts(positions.loc[slice(None),(slice(None),player,"BP")][positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]
        prozanteil=(anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID2,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)

6.2110616656071205
7.980504344140709
11.351981351981353
12.566221657130749
13.524051705869889
15.977961432506888
16.429328247510067
20.129264674719224
21.551176096630645
22.924348378893836
26.965458783640603
28.51239669421488
33.94363212545031
38.713710531892346
45.386734477643564
46.10510701419792
47.41894469167196
49.18203009112099
50.271243907607534
54.86967577876668
56.95062513244331
58.45941936851027


In [ ]:
#simplester ansatz
summe=0
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        anzframes=pd.value_counts(positions2.loc[slice(None),(slice(None),player,"BP")][positions2.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1].values.flatten())[1]
        prozanteil=(anzframes*100/((positions2.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID1,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)
    if player in matchinfo.get(2)[CUID2]:
        anzframes=pd.value_counts(positions2.loc[slice(None),(slice(None),player,"BP")].values.flatten())[1]
        prozanteil=(anzframes*100/((positions2.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        summe=summe+prozanteil
        #print(CUID2,player,anzframes*100/((positions.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1)).sum())
        print(summe)

In [152]:
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        anzframes=pd.value_counts(positions2.loc[slice(None),(slice(None),player,"BP")].values.flatten())[1]
        print(CUID1,player,anzframes*100/positions2.shape[0])
    if player in matchinfo.get(2)[CUID2]:
        anzframes=pd.value_counts(positions2.loc[slice(None),(slice(None),player,"BP")].values.flatten())[1]
        print(CUID2,player,anzframes*100/positions2.shape[0])

DFL-CLU-000N8Y DFL-OBJ-000191 3.5432624113475177
DFL-CLU-000N8Y DFL-OBJ-0027G6 1.2156028368794327
DFL-CLU-000N8Z DFL-OBJ-002FZX 1.724822695035461
DFL-CLU-000N8Z DFL-OBJ-002G01 0.5588652482269504
DFL-CLU-000N8Y DFL-OBJ-0000I4 1.3333333333333333
DFL-CLU-000N8Z DFL-OBJ-002FZQ 0.20141843971631207
DFL-CLU-000N8Z DFL-OBJ-002FZP 0.13191489361702127
DFL-CLU-000N8Z DFL-OBJ-002G03 0.4453900709219858
DFL-CLU-000N8Z DFL-OBJ-002G08 0.8666666666666667
DFL-CLU-000N8Y DFL-OBJ-0001HT 1.9078014184397163
DFL-CLU-000N8Y DFL-OBJ-0002AU 1.852482269503546
DFL-CLU-000N8Y DFL-OBJ-0000OJ 0.475177304964539
DFL-CLU-000N8Y DFL-OBJ-0000ZS 3.2099290780141843
DFL-CLU-000N8Y DFL-OBJ-0002F5 6.502127659574468
DFL-CLU-000N8Z DFL-OBJ-002FZY 0.08085106382978724
DFL-CLU-000N8Z DFL-OBJ-002G05 0.26382978723404255
DFL-CLU-000N8Y DFL-OBJ-0000O3 1.0510638297872341
DFL-CLU-000N8Z DFL-OBJ-002FZW 0.25957446808510637
DFL-CLU-000N8Y DFL-OBJ-00258K 1.8439716312056738
DFL-CLU-000N8Z DFL-OBJ-002FZZ 0.36028368794326243
DFL-CLU-000N8Z DFL

In [201]:
positions.loc[slice(48140,49190),("BALL","DFL-OBJ-0000XT","Z")]

N      T                               
48140  2019-06-11 21:12:27.200000+02:00    0.00
48141  2019-06-11 21:12:27.240000+02:00    0.00
48142  2019-06-11 21:12:27.280000+02:00    0.00
48143  2019-06-11 21:12:27.320000+02:00    0.00
48144  2019-06-11 21:12:27.360000+02:00    0.00
48145  2019-06-11 21:12:27.400000+02:00    0.00
48146  2019-06-11 21:12:27.440000+02:00    0.00
48147  2019-06-11 21:12:27.480000+02:00    0.00
48148  2019-06-11 21:12:27.520000+02:00    0.00
48149  2019-06-11 21:12:27.560000+02:00    0.00
48150  2019-06-11 21:12:27.600000+02:00    0.00
48151  2019-06-11 21:12:27.640000+02:00    0.00
48152  2019-06-11 21:12:27.680000+02:00    0.07
48153  2019-06-11 21:12:27.720000+02:00    0.14
48154  2019-06-11 21:12:27.760000+02:00    0.21
48155  2019-06-11 21:12:27.800000+02:00    0.28
48156  2019-06-11 21:12:27.840000+02:00    0.34
48157  2019-06-11 21:12:27.880000+02:00    0.41
48158  2019-06-11 21:12:27.920000+02:00    0.48
48159  2019-06-11 21:12:27.960000+02:00    0.55


In [65]:
positions.loc[slice(15650,15710),(slice(None),"DFL-OBJ-0002AU","DtB")]

,TeamId,DFL-CLU-000N8Y
,PersonId,DFL-OBJ-0002AU
,Position,DtB
N,T,
15650,2019-06-11 20:50:47.600000+02:00,21.250085
15651,2019-06-11 20:50:47.640000+02:00,21.190340
15652,2019-06-11 20:50:47.680000+02:00,21.130606
15653,2019-06-11 20:50:47.720000+02:00,21.091254
15654,2019-06-11 20:50:47.760000+02:00,21.041863
15655,2019-06-11 20:50:47.800000+02:00,20.982755
15656,2019-06-11 20:50:47.840000+02:00,20.933822


In [199]:
kpimergednew.loc[:,"SUBTYPE"].value_counts()

Pass                 1250
OtherBallAction       164
Ground                110
BallClaiming           26
ThrowIn                23
Air                    22
Cross                  21
GoalKick               15
FreeKick               14
SavedShot              11
CornerKick             11
Kickoff                10
Foul                   10
ShotWide                8
SuccessfulShot          8
Substitution            6
Offside                 5
BlockedShot             3
OtherShot               3
OtherPlayerAction       1
Caution                 1
Penalty                 1
Name: SUBTYPE, dtype: int64

In [57]:
kpimergednew[(kpimergednew.loc[:,"SUBTYPE"]=="Pass")].loc[:,"EVALUATION"].value_counts()

successfullyComplete    1123
unsuccessful             126
successful                 1
Name: EVALUATION, dtype: int64

In [61]:
passseskpi=kpimergednew[(kpimergednew.loc[:,"SUBTYPE"]=="Pass")]

In [62]:
passseskpi[(passseskpi.loc[:,"EVALUATION"]=="successfullyComplete")].loc[:,"NORECEIVER"].value_counts()

False    1107
True       16
Name: NORECEIVER, dtype: int64

# Durchschnittliche Zeit von Annahme bis Abspiel pro individuellem Ballbesitz

In [ ]:
#RESET

In [202]:
##Placeholder Column ob in Ballbesitz oder nicht
for player in players:
    if player in matchinfo.get(1)[CUID1]:
        positions[CUID1,player,'BP']=0
    if player in matchinfo.get(1)[CUID2]:
        positions[CUID2,player,'BP']=0
        
for player in players:
    if player in matchinfo.get(2)[CUID1]:
        positions2[CUID1,player,'BP']=0
    if player in matchinfo.get(2)[CUID2]:
        positions2[CUID2,player,'BP']=0

In [ ]:
##Dataframe erstellen 

In [ ]:
BPDF = {CUID1: [passdf1.shape[0],passsuccessrecdf1.shape[0],passunsuccessdf1.shape[0],passsuccessrecdf1['qpass90'].value_counts()[1],passsuccessrecdf1['qpass90'].value_counts()[0],cuid190ges,passsuccessrecdf1['qpass90goal'].value_counts()[1],passsuccessrecdf1['qpass90goal'].value_counts()[0],cuid190goalges],
                 CUID2: [passdf2.shape[0],passsuccessrecdf2.shape[0],passunsuccessdf2.shape[0],passsuccessrecdf2['qpass90'].value_counts()[1],passsuccessrecdf2['qpass90'].value_counts()[0],cuid290ges,passsuccessrecdf2['qpass90goal'].value_counts()[1],passsuccessrecdf2['qpass90goal'].value_counts()[0],cuid290goalges]}
passstatisticdf = pd.DataFrame(passstatistic,columns= [CUID1, CUID2],
                 index=['Total','Successful','Unsuccessful','Successful in Spielrichtung','Successful gegen Spielrichtung','Total Pässe in Spielrichtung','Successful in Spielrichtung (Tormitte)','Successful gegen Spielrichtung (Tormitte)','Total in Spielrichtung (Tormitte)'])
print(passstatisticdf)


In [205]:
import datetime

In [211]:
#todays_date = datetime.datetime.now().date()
#index = pd.date_range(todays_date-datetime.timedelta(10), periods=10, freq='D')
index=range(1000)
columns=matchinfo.get(1)[CUID1]
CUID1BPDF1 = pd.DataFrame(index=index, columns=columns)
CUID1BPDF1 = CUID1BPDF1.fillna(0)


columns=matchinfo.get(1)[CUID2]
CUID2BPDF1 = pd.DataFrame(index=index, columns=columns)
CUID2BPDF1 = CUID1BPDF1.fillna(0)

columns=matchinfo.get(2)[CUID1]
CUID1BPDF2 = pd.DataFrame(index=index, columns=columns)
CUID1BPDF2 = CUID1BPDF1.fillna(0)

columns=matchinfo.get(2)[CUID2]
CUID2BPDF2 = pd.DataFrame(index=index, columns=columns)
CUID2BPDF2 = CUID1BPDF1.fillna(0)

In [76]:
dct = {}
for player in players:
    player=player.replace('-', '')
    dct['BPDF_%s' % player] = []
print(dct)

{'BPDF_DFLOBJ000191': [], 'BPDF_DFLOBJ0027G6': [], 'BPDF_DFLOBJ002FZX': [], 'BPDF_DFLOBJ002G01': [], 'BPDF_DFLOBJ0000I4': [], 'BPDF_DFLOBJ002FZQ': [], 'BPDF_DFLOBJ002FZP': [], 'BPDF_DFLOBJ0000SE': [], 'BPDF_DFLOBJ002G03': [], 'BPDF_DFLOBJ002G08': [], 'BPDF_DFLOBJ0001HT': [], 'BPDF_DFLOBJ0002AU': [], 'BPDF_DFLOBJ0000OJ': [], 'BPDF_DFLOBJ0000ZS': [], 'BPDF_DFLOBJ0002F5': [], 'BPDF_DFLOBJ002FZY': [], 'BPDF_DFLOBJ002G05': [], 'BPDF_DFLOBJ0000O3': [], 'BPDF_DFLOBJ002FZW': [], 'BPDF_DFLOBJ00258K': [], 'BPDF_DFLOBJ002FZZ': [], 'BPDF_DFLOBJ002G0B': [], 'BPDF_DFLOBJ0001F4': [], 'BPDF_DFLOBJ002G0E': [], 'BPDF_DFLOBJ0029Y8': [], 'BPDF_DFLOBJ002FZR': [], 'BPDF_DFLOBJ00021U': [], 'BPDF_DFLOBJ0000Q9': []}


In [22]:
x=7
dct.get("BPDF_DFLOBJ000191").append(x)
dct.get("BPDF_DFLOBJ000191")

[7]

In [26]:
kpimergednew.loc[12,"PUID2"].replace('-', '')

'DFLOBJ00258K'

In [23]:
dct.get("BPDF_{}".format(kpimergednew.loc[12,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"N"])

[]

In [ ]:
STAND JETZT

In [31]:
kpimergednew.loc[0,"PUID2"].replace('-', '')

AttributeError: 'float' object has no attribute 'replace'

In [32]:
i=0
kpimergednew.loc[i,"SUBTYPE"]=="Pass" and kpimergednew.loc[i,"NORECEIVER"]==False and kpimergednew.loc[i,"EVALUATION"]=="successfullyComplete"

False

In [41]:
#Durchschnittliche Dauer
#1st Try
for i in range(kpimergednew.shape[0]-1):
    #print(kpimergednew.loc[i,"N"])
    if kpimergednew.loc[i,"SUBTYPE"]=="Pass" and kpimergednew.loc[i,"NORECEIVER"]==False and kpimergednew.loc[i,"EVALUATION"]=="successfullyComplete":
        ##falls Event Pass und es einen abhnehmer gibt, der im eigenen Team ist
            ##Überprüfung welche Halbzeit
        if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
            #positions.loc[slice(kpimergednew.loc[i-1,"N"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
            #dauer1=kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"N"]
            #player1=kpimergednew.loc[i,"PUID"]
            dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"N"])
            ## berechnet die zeitdif, nimmt sich den spieler und hängt den wert an seine liste
                
            #positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID2"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"RECFRM"])
                
        if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
            #positions.loc[slice(kpimergednew.loc[i-1,"RECFRM"],kpimergednew.loc[i,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID1"],"BP")]=1
            dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"RECFRM"])
                
            #positions.loc[slice(kpimergednew.loc[i,"RECFRM"],kpimergednew.loc[i+1,"N"]),(kpimergednew.loc[i,"CUID"],kpimergednew.loc[i,"PUID2"],"BP")]=1
            dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID2"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"RECFRM"])
                           
   # if kpimergednew.loc[i,"SUBTYPE"]!="Pass" and kpimergednew.loc[i,"SUBTYPE"]!="Substitution":

        #positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
       # dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"N"])
    if kpimergednew.loc[i,"SUBTYPE"]=="BallClaiming":
        dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"N"])   
        #positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
    
    if kpimergednew.loc[i,"SUBTYPE"]=="Pass" and kpimergednew.loc[i,"EVALUATION"]=="unsuccessful":
        
        if kpimergednew.loc[i-1,"SUBTYPE"]=="Pass" and kpimergednew.loc[i-1,"NORECEIVER"]==False and kpimergednew.loc[i-1,"EVALUATION"]=="successfullyComplete":
        
        dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"N"])   

In [77]:
#Durchschnittliche Dauer
#2nd TRY
for i in range(1,kpimergednew.shape[0]-1):
    #print(kpimergednew.loc[i,"N"])
   
        
    if kpimergednew.loc[i,"SUBTYPE"]=="Pass":            
        
        
        if kpimergednew.isnull().loc[i-1,"RECFRM"]==True:
            
            if kpimergednew.loc[i,"N"]<matchinfo.get(2)["Startframe"]:
                
                if (positions.loc[kpimergednew.loc[i-1,"N"]:kpimergednew.loc[i,"N"],("BALL","DFL-OBJ-0000XT","BallStatus")]==1).value_counts(True)[True]>0.95:
                    
                    dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"N"])
            else:
                ###AKTUELLER STAND
                if (positions2.loc[kpimergednew.loc[i-1,"N"]:kpimergednew.loc[i,"N"],("BALL","DFL-OBJ-0000XT","BallStatus")]==1).value_counts(True)[True]>0.95:
                    
                    dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"N"])
            
        if kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
            
            #dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"RECFRM"])
            
            if kpimergednew.loc[i,"PUID1"]==kpimergednew.loc[i-1,"PUID2"]:
                
                dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"RECFRM"])
    
    
    ##Überprüft ob im Frame vorher der Spieler einen Pass erhalten hat
    if kpimergednew.loc[i,"SUBTYPE"]!="Pass" and kpimergednew.isnull().loc[i-1,"RECFRM"]==False:
            
        if kpimergednew.loc[i,"PUID1"]==kpimergednew.loc[i-1,"PUID2"]:
            
            if kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"RECFRM"]<=0:
                dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(1)
            else:
                dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i,"N"]-kpimergednew.loc[i-1,"RECFRM"])
    
    if kpimergednew.loc[i,"SUBTYPE"]=="BallClaiming":
            
        if kpimergednew.loc[i+1,"PUID1"]==kpimergednew.loc[i,"PUID1"]:
            
            dct.get("BPDF_{}".format(kpimergednew.loc[i,"PUID1"].replace('-', ''))).append(kpimergednew.loc[i+1,"N"]-kpimergednew.loc[i,"N"])   
        #positions.loc[slice(kpimergednew.loc[i,"N"],kpimergednew.loc[i+1,"N"]),(slice(None),kpimergednew.loc[i,"PUID1"],"BP")]=1
    

In [78]:

dct.get("BPDF_DFLOBJ002G05")

[35, 0, 9.0, 38.0, 17, 50, 1.0, 16.0, 178, 36, 46.0, 0, 90.0, 46.0]

In [43]:
sum(dct.get("BPDF_DFLOBJ002G05")) / len(dct.get("BPDF_DFLOBJ002G05")) 

39.333333333333336

In [52]:
summe=0
laenge=0
for player in players:
    if player in matchinfo.get(1)[CUID1] or player in matchinfo.get(2)[CUID1]:
        
        print(sum( dct.get("BPDF_{}".format(player.replace('-', '')))) / len(dct.get("BPDF_{}".format(player.replace('-', '')))) , len(dct.get("BPDF_{}".format(player.replace('-', '')))))
        summe=summe+ sum( dct.get("BPDF_{}".format(player.replace('-', ''))))
        laenge=laenge+len(dct.get("BPDF_{}".format(player.replace('-', ''))))
        
summe/laenge

32.778688524590166 244
206.6638655462185 119
41.479166666666664 48
28.455696202531644 79
27.80246913580247 162
32.19767441860465 86
20.080188679245282 212
29.31578947368421 228
40.323232323232325 297
13.77319587628866 97
37.02547770700637 157
38.96551724137931 58
42.40909090909091 22
83.0 87


44.61128691983122

In [51]:
summe=0
laenge=0
for player in players:
    if player in matchinfo.get(1)[CUID2] or player in matchinfo.get(2)[CUID2]:
        
        print(sum( dct.get("BPDF_{}".format(player.replace('-', '')))) / len(dct.get("BPDF_{}".format(player.replace('-', '')))) , len(dct.get("BPDF_{}".format(player.replace('-', '')))))
        summe=summe+ sum( dct.get("BPDF_{}".format(player.replace('-', ''))))
        laenge=laenge+len(dct.get("BPDF_{}".format(player.replace('-', ''))))
        
summe/laenge

149.375 24
40.6 30
27.09090909090909 33
12.142857142857142 14
37.62222222222222 45
45.15 40
22.130434782608695 23
39.333333333333336 18
37.44444444444444 18
32.916666666666664 48
13.607142857142858 28
9.6 5
30.0 6
25.75 12


39.98546511627907

In [79]:
summe=0
laenge=0
for player in players:
    if player in matchinfo.get(1)[CUID1] or player in matchinfo.get(2)[CUID1]:
        
        print(sum( dct.get("BPDF_{}".format(player.replace('-', '')))) / len(dct.get("BPDF_{}".format(player.replace('-', '')))) , len(dct.get("BPDF_{}".format(player.replace('-', '')))))
        summe=summe+ sum( dct.get("BPDF_{}".format(player.replace('-', ''))))
        laenge=laenge+len(dct.get("BPDF_{}".format(player.replace('-', ''))))
        
summe/laenge

32.70454545454545 132
358.2878787878788 66
54.3 30
33.67391304347826 46
31.211111111111112 90
33.27450980392157 51
21.908256880733944 109
32.016806722689076 119
44.39102564102564 156
18.115384615384617 52
36.455555555555556 90
42.81818181818182 33
51.214285714285715 14
75.27659574468085 47


56.678260869565214

In [80]:
summe=0
laenge=0
for player in players:
    if player in matchinfo.get(1)[CUID2] or player in matchinfo.get(2)[CUID2]:
        
        print(sum( dct.get("BPDF_{}".format(player.replace('-', '')))) / len(dct.get("BPDF_{}".format(player.replace('-', '')))) , len(dct.get("BPDF_{}".format(player.replace('-', '')))))
        summe=summe+ sum( dct.get("BPDF_{}".format(player.replace('-', ''))))
        laenge=laenge+len(dct.get("BPDF_{}".format(player.replace('-', ''))))
        
summe/laenge

130.78787878787878 33
46.0 19
32.15384615384615 26
12.454545454545455 11
37.52 25
38.233333333333334 30
23.615384615384617 13
40.142857142857146 14
25.58823529411765 17
37.70967741935484 31
19.5 24
28.6 5
35.4 5
29.666666666666668 9


44.94656488549618

In [105]:
(positions2.loc[:,("BALL","DFL-OBJ-0000XT","BallStatus")]==1).value_counts(True)[True]

0.6595319148936171

In [104]:
matchinfo.get(2)["Startframe"]

100000